The following piece of code supports the AI model KIMI, which is based on OpenAI. I used this model in the script because I couldn't create an OpenAI API key with a Chinese phone number.

Create an api key in "https://platform.moonshot.cn/console/api-keys", then create Environment variable.

Prompts and accuracy:
1. please carefully read the text and the following question, pick the correct one from the given options:   87.00
2. Choose the right option based on the information provided:  85.00
3. Find the correct option among the following choices:   85.00
4. Please select the correct option based on the information provided：  84.00
5. Your task is to choose the correct option based on the following description:    84.00
6. Select the most appropriate answer from the options:  84.00
7. Please select the correct answer:   84.00
8. Based on the details, what is the correct answer?   82.00
9. Answer the question:   81.00
10. Please answer the following question:  73.00
11. 根据提供的信息选择正确的选项：  83.00
12. 根据给出的细节，找出正确的答案： 80.00
13. 请回答问题：   72.00

In [17]:
import sys
import os
import openai
import datasets
import asyncio
import nest_asyncio
import argparse
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

def get_prompt_template():
  print("Enter a prompt:")
  user_input = input().strip()
  if not user_input:
    user_input = """Answer the question: """
  print(user_input)
  return user_input

# get response from AI model
async def get_llm_response_async(prompt, prompt_template, model):
  formatted_prompt = prompt_template + prompt
  openai_api_key = os.environ.get("OPENAI_API_KEY")
  #print(f"openai_api_key:{openai_api_key}")
  client = openai.AsyncOpenAI(api_key=openai_api_key,
                              base_url="https://api.moonshot.cn/v1"
                              )
  
  response = await client.chat.completions.create(
      model=model,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": formatted_prompt}
      ],
      max_tokens=500,  # Adjust as needed
      n=1, 
      stop=None,
      temperature=0.0, #0-1
  )

  return response.choices[0].message.content.strip()

def evaluate(response, target):
  response_label = response.split()[0].strip("() ")  # Extract the answer label
  target_label = target.strip("() ")
  return response_label == target_label

async def main(args):

  dataset = datasets.load_dataset("lukaemon/bbh", "reasoning_about_colored_objects")
  subset = dataset["test"].shuffle(seed=args.seed).select(range(args.limit))
  # for item in subset:
  #   print(item)
  # example : {"input":xxxx,"target":(F)}

  prompt_template = get_prompt_template() # user_input or "Answer the question:"

  semaphore = asyncio.Semaphore(args.concurrency)

  from pprint import pprint
  async def process_example(example):
    async with semaphore:
      input_ = example["input"]
      target = example["target"]
      response = await get_llm_response_async(input_, prompt_template, args.model)
      is_correct = evaluate(response, target)
      result = {
        'question': input_,
        'ground_truth': target,
        'model_response': response,
        'is_correct': is_correct
      }
      # if not is_correct:
      #   print(result)
      return result

  tasks = [process_example(example) for example in subset] # result list

  evaluation_results = await tqdm_asyncio.gather(*tasks, desc="Processing examples")
 
  # Calculate and print the metrics
  accuracy = 0
  # TODO: calculate accuracy
  example_pass = sum(1 for res in evaluation_results if res["is_correct"])
  accuracy = example_pass / len(tasks) * 100
  print(f"Accuracy: {accuracy:.2f}")
  

  # TODO: implement any other metric that you think might be usefull

if __name__ == "__main__":
  args = argparse.ArgumentParser()
  args.add_argument("--limit", type=int, default=100) # limit 100 examples
  #args.add_argument("--model", type=str, default="gpt-3.5-turbo")
  #models "moonshot-v1-8k","moonshot-v1-32k","moonshot-v1-128k"
  args.add_argument("--model", type=str, default="moonshot-v1-8k")
  args.add_argument("--seed", type=int, default=42)
  args.add_argument("--concurrency", type=int, default=5)
  
  args = args.parse_known_args()[0]
  
  asyncio.run(main(args))
  

Enter a prompt:
请回答问题：


Processing examples: 100%|██████████| 100/100 [01:04<00:00,  1.56it/s]

Accuracy: 72.00


The following piece of code supports openai model, I haven't actually run it myself because I wasn't able to create a openai api key using a Chinese phone number. I believe it will work well with a valid openai api key. 

In [ ]:
import sys
import os
import openai
import datasets
import asyncio
import nest_asyncio
import argparse
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

def get_prompt_template():
  print("Enter a prompt:")
  user_input = input().strip()
  if not user_input:
    user_input = """Answer the question: """
  print(user_input)
  return user_input

# get response from AI model
async def get_llm_response_async(prompt, prompt_template, model):
  formatted_prompt = prompt_template + prompt
  openai_api_key = os.environ.get("OPENAI_API_KEY")
  client = openai.AsyncOpenAI(api_key=openai_api_key)
  
  response = await client.chat.completions.create(
      model=model,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": formatted_prompt}
      ],
      max_tokens=500,  # Adjust as needed
      n=1, 
      stop=None,
      temperature=0.0, #0-1
  )

  return response.choices[0].message.content.strip()

def evaluate(response, target):
  response_label = response.split()[0].strip("() ")  # Extract the answer label
  target_label = target.strip("() ")
  return response_label == target_label

async def main(args):

  dataset = datasets.load_dataset("lukaemon/bbh", "reasoning_about_colored_objects")
  subset = dataset["test"].shuffle(seed=args.seed).select(range(args.limit))
  # for item in subset:
  #   print(item)
  # example : {"input":xxxx,"target":(F)}

  prompt_template = get_prompt_template() 

  semaphore = asyncio.Semaphore(args.concurrency)

  from pprint import pprint
  async def process_example(example):
    async with semaphore:
      input_ = example["input"]
      target = example["target"]
      response = await get_llm_response_async(input_, prompt_template, args.model)
      is_correct = evaluate(response, target)
      result = {
        'question': input_,
        'ground_truth': target,
        'model_response': response,
        'is_correct': is_correct
      }
      # if not is_correct:
      #   print(result)
      return result

  tasks = [process_example(example) for example in subset] # result list

  evaluation_results = await tqdm_asyncio.gather(*tasks, desc="Processing examples")
 
  # Calculate and print the metrics
  accuracy = 0
  # TODO: calculate accuracy
  example_pass = sum(1 for res in evaluation_results if res["is_correct"])
  accuracy = example_pass / len(tasks) * 100
  print(f"Accuracy: {accuracy:.2f}")
  

  # TODO: implement any other metric that you think might be usefull

if __name__ == "__main__":
  args = argparse.ArgumentParser()
  args.add_argument("--limit", type=int, default=100) # limit 100 examples
  args.add_argument("--model", type=str, default="gpt-3.5-turbo")
  args.add_argument("--seed", type=int, default=42)
  args.add_argument("--concurrency", type=int, default=5)
  
  args = args.parse_known_args()[0]
  
  asyncio.run(main(args))
  



The following piece of code supports Alibaba model "qwen"

prompts：
1. 请根据描述的内容回答问题，给出正确选项的编号即可，不需要其他信息   86.00
2. Answer the question:  94.00
3. 请回答问题：   96.00
4. 请根据给出的描述信息回答问题，选择最正确的选项，返回选项的编号：    84.00

In [16]:
import sys
import os
import openai
import datasets
import asyncio
import nest_asyncio
import argparse
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

def get_prompt_template():
  print("Enter a prompt:")
  user_input = input().strip()
  if not user_input:
    user_input = """Answer the question: """
  print(user_input)
  return user_input

# get response from AI model
async def get_llm_response_async(prompt, prompt_template, model):
  formatted_prompt = prompt_template + prompt
  alibaba_api_key = os.getenv("DASHSCOPE_API_KEY")
  #print(f"ALIBABA API key:{alibaba_api_key}")
  client = openai.AsyncOpenAI(api_key=alibaba_api_key,
                              base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
                              )
  
  response = await client.chat.completions.create(
      model=model,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": formatted_prompt}
      ],
      max_tokens=500,  # Adjust as needed
      n=1, 
      stop=None,
      temperature=0.0, #0-1
  )

  return response.choices[0].message.content.strip()
import re
def evaluate(response, target):
  # response_label = response.split()[0].strip("() ")
  # taget_label = taget.strip("() ")
  
  response_label = re.search(r'\(([A-Z]{1})\)',response)
  if response_label:
      response_label = response_label.group(0)
      return target == response_label
  
  #print(f"response_label:{response_label}; target_lebal:{target}")

  return False 
  

async def main(args):

  dataset = datasets.load_dataset("lukaemon/bbh", "reasoning_about_colored_objects")
  subset = dataset["test"].shuffle(seed=args.seed).select(range(args.limit))
  # for item in subset:
  #   print(item)
  # example : {"input":xxxx,"target":(F)}

  prompt_template = get_prompt_template() # user_input or "Answer the question:"

  semaphore = asyncio.Semaphore(args.concurrency)

  from pprint import pprint
  async def process_example(example):
    async with semaphore:
      input_ = example["input"]
      target = example["target"]
      response = await get_llm_response_async(input_, prompt_template, args.model)
      is_correct = evaluate(response, target)
      result = {
        'question': input_,
        'ground_truth': target,
        'model_response': response,
        'is_correct': is_correct
      }
      # if not is_correct:
      #   print(result)
      return result

  tasks = [process_example(example) for example in subset] # result list

  evaluation_results = await tqdm_asyncio.gather(*tasks, desc="Processing examples")
 
  # Calculate and print the metrics
  accuracy = 0
  # TODO: calculate accuracy
  example_pass = sum(1 for res in evaluation_results if res["is_correct"])
  accuracy = example_pass / len(tasks) * 100
  print(f"Accuracy: {accuracy:.2f}")
  

  # TODO: implement any other metric that you think might be usefull

if __name__ == "__main__":
  args = argparse.ArgumentParser()
  args.add_argument("--limit", type=int, default=100) # limit 100 examples
  args.add_argument("--model", type=str, default="qwen-plus")
  args.add_argument("--seed", type=int, default=42)
  args.add_argument("--concurrency", type=int, default=5)
  
  args = args.parse_known_args()[0]
  
  asyncio.run(main(args))

Enter a prompt:
请回答问题：


Processing examples: 100%|██████████| 100/100 [02:47<00:00,  1.67s/it]

Accuracy: 96.00
